In [15]:
import numpy as np
import random
import pandas as pd

In [4]:
v_list=[-2,-1,1,2] #possible values for the bjs and bzs
test_con_list=dict() #context list
test_con_list["has_social"]=2
test_con_list['question_bucket']=4
#test_con_list
test_att={
    'gender':2,
    'case_note':3,
    'race':3,
    'length':4,
    'age':4,
    'setting':4,
    'goal':5,
    'n_removal':3,
    'reason':5
}
bjs_sparsity_list=[.1, 0.5,.9] #[0,1]
bjs_multiplier_list=[1,5] #[1,10]
bzs_sparsity_list=[.1, 0.5,.9] #[0,1]
bzs_multiplier_list=[1,5] #[1,10]

In [12]:
def gen_clist(con_dict,n_response): #generate context list
    empty=[]
    count=0
    for i in con_dict.values():
        empty=np.concatenate([empty,np.random.choice(i,n_response)])
        count+=1
    #print(empty)
    empty=np.array(empty)
    #print(empty)
    return empty.reshape(count,n_response).T
def oh_c(gen_c,con_dict): #one hot encode context list
    final=[]
    con_v=[x for x in con_dict.values()]
    #print(con_v)
    for i in gen_c:
        temp=[]
        for j in range(len(con_v)):
            temp=np.concatenate([temp,np.eye(con_v[j])[int(i[j])]])
        final.append(temp)
    return final
def gen_sparsity(n,sparsity): #generate sparse lists/matrcies
    return np.random.choice(n,int(n*sparsity),replace=False)#generate sparse indices
def gen_base_bjs(v_list,sum_count): #generate the baseline level of bjs (since we have different sparsity and multiplers in a sim)
    return np.random.choice(v_list, sum_count)
def gen_bjs(unsparse,sum_count,sparsity,multiplier): #take the base_bjs then alter it base on spar and mult
    #unsparse=np.random.choice(v_list, sum_count)
    spar=gen_sparsity(sum_count,sparsity)
    for i in spar:
        unsparse[i]=0
    return unsparse*multiplier
def fix_first_level_bjs(base_bj, test_att): #setting the first level of bjs to 0
    #acc=[]
    count=0
    for i in [x for x in test_att.values()]:
        #acc.append(count)
        base_bj[count]=0
        count+=i
    return base_bj
def fix_first_level_context(base_bz,test_con): #setting the first level of context list to 0
    #count=0
    for i in base_bz:
        count=0
        for j in [x for x in test_con_list.values()]:
            i[count]=0
            count +=j
    return base_bz
def gen_base_bzs(v_list, sum_count, con_sum): #generate the baseline level of bzs
    return np.random.choice(v_list, sum_count*con_sum)
def gen_bzs(unsparse,sum_count,con_sum,sparsity,multiplier): #alter bzs
    #unsparse=np.random.choice(v_list, sum_count*con_sum)
    spar=gen_sparsity(sum_count*con_sum,sparsity)
    for i in spar:
        unsparse[i]=0
    return unsparse.reshape(sum_count,con_sum)*multiplier
def gen_one_profile(c_list): #generate one profile
    final=[]
    og=[]
    for i in c_list: #attribute list, so for each attribute generate a random number that is within the constraint of it
        random_int=np.random.randint(i)
        #print(random_int)
        final=np.concatenate([final,np.eye(i)[random_int]])
        og.append(random_int)
    return final, og
def gen_profiles(n,n_trial,count_list):
    final_n_dim_array=[] 
    for i in range(n):
        ten_questions=[]
        og=[]
        for j in range(n_trial):
            first_prof=gen_one_profile(count_list)
            second_prof=gen_one_profile(count_list)
            ten_questions.append([first_prof[0],second_prof[0]])
            #print(ten_questions)
            og.append([first_prof[1],second_prof[1]])
        final_n_dim_array.append([ten_questions,og])
    return final_n_dim_array
def all_sim(n_response, n_trial, context_dict, attribute_dict, #generate simulations
            bjs_sparsity_list, bjs_multiplier_list, bzs_sparsity_list, 
            bzs_multiplier_list, n_sim_dataset,v_list):
    data=[]
    for sim_id in range(n_sim_dataset):
        context_list=gen_clist(context_dict,n_response)
        one_hot_con_list=oh_c(context_list,context_dict)
        profiles=gen_profiles(n_response,n_trial,attribute_dict.values())
        base_bjs=gen_base_bjs(v_list, sum(attribute_dict.values()))
        base_bjs=fix_first_level_bjs(base_bjs,attribute_dict)#fixing first levels of bjs
        base_bzs=gen_base_bzs(v_list, sum(attribute_dict.values()), sum(context_dict.values()))
        #base_bzs=fix_first_level_context(base_bzss,context_dict)
        for bjs_multiplier in bjs_multiplier_list:
            sigma_r=np.random.uniform(0.01,bjs_multiplier)
            for bjs_sparsity in bjs_sparsity_list:
                for bzs_sparsity in bzs_sparsity_list:
                    for bzs_multiplier in bzs_multiplier_list:
                        bjs=gen_bjs(base_bjs,sum(attribute_dict.values()),bjs_sparsity,bjs_multiplier)
                        bzs=gen_bzs(base_bzs, sum(attribute_dict.values()),
                                    sum(context_dict.values()),bzs_sparsity,bzs_multiplier)
                        bzs=fix_first_level_context(bzs,context_dict)
                        #sigma_r=np.random.uniform(0.01,bjs_multiplier) 
                        for response in range(n_response):
                            context=one_hot_con_list[response]
                            context_og=context_list[response]
                            profile=profiles[response][0]
                            profile_og=profiles[response][1]
                            for trial in range(n_trial):
                                prof_trial=profile[trial]
                                prof_trial_og=profile_og[trial]
                                mu0=np.dot(prof_trial[0],bjs)+np.dot(np.dot(prof_trial[0],bzs),context)
                                y0=np.random.normal(mu0,sigma_r)

                                mu1=np.dot(prof_trial[1],bjs)+np.dot(np.dot(prof_trial[1],bzs),context)
                                y1=np.random.normal(mu1,sigma_r)
                                bernoulli=1
                                if y0>y1:
                                    bernoulli=0 #pretty sure we concluded the meeting w isys stating it should be deterministic
                                data.append([sim_id, response, context, context_og, prof_trial[0], 
                                             np.array(prof_trial_og[0]),1-bernoulli, 
                                            bjs_sparsity, bjs_multiplier, bzs_sparsity, bzs_multiplier, sigma_r,
                                             bjs,bzs
                                            ])
                                data.append([sim_id, response, context, context_og, prof_trial[1],
                                             np.array(prof_trial_og[1]),bernoulli, 
                                            bjs_sparsity, bjs_multiplier, bzs_sparsity, bzs_multiplier, sigma_r,
                                             bjs,bzs
                                            ])
    return data


In [13]:
test_all=all_sim(400,10,test_con_list,test_att,bjs_sparsity_list, bjs_multiplier_list, bzs_sparsity_list, 
            bzs_multiplier_list,30,v_list)

In [7]:
#unwrap columns with arrays into one element per column
def unwrap_prof(df,att_dict): 
    col_count=0
    for i in att_dict.items():
        for j in range(i[1]):
            col_name=i[0]+str(j)
            def get(pu):
                return pu['profile'][col_count]
            df[col_name]=df.apply(get, axis=1)
            col_count +=1
    return df
def unwrap_prof_og(df,att_dict):
    col_count=0
    for i in att_dict.items():        
        col_name=i[0]
        def get(pu):
            return pu['profile_og'][col_count]
        df[col_name]=df.apply(get, axis=1)
        col_count +=1
    return df
def unwrap_context(df,att_dict):
    col_count=0
    for i in att_dict.items():
        for j in range(i[1]):
            col_name=i[0]+str(j)
            def get(pu):
                return pu['context'][col_count]
            df[col_name]=df.apply(get, axis=1)
            col_count +=1
    return df
def unwrap_context_og(df,att_dict):
    col_count=0
    for i in att_dict.items():

            col_name=i[0]
            def get(pu):
                return pu['context_og'][col_count]
            df[col_name]=df.apply(get, axis=1)
            col_count +=1
    return df

In [16]:
tp=np.array(test_all,dtype=object)
tp=tp.transpose()
df=pd.DataFrame({'sim_id':tp[0],
                'respondent_id':tp[1],
                 'context':tp[2],
                 'context_og':tp[3],
                 'profile':tp[4],
                 'profile_og':tp[5],
                 'selected':tp[6],
                 'bjs_spar':tp[7],
                 'bjs_mul':tp[8],
                 'bzs_spar':tp[9],
                 'bzs_mul':tp[10],
                 'sigma_r':tp[11],
                 'bjs':tp[12],
                 'bzs':tp[13]
                })

In [17]:
unwrap_prof(df, test_att)
print(1)
unwrap_prof_og(df, test_att)
print(2)
unwrap_context(df, test_con_list)
print(3)
unwrap_context_og(df, test_con_list)

1
2
3


,sim_id,respondent_id,context,context_og,profile,profile_og,selected,bjs_spar,bjs_mul,bzs_spar,...,n_removal,reason,has_social0,has_social1,question_bucket0,question_bucket1,question_bucket2,question_bucket3,has_social,question_bucket
0,0,0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 2.0]","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[1, 1, 2, 0, 2, 1, 0, 1, 4]",0,0.1,1,0.1,...,1,4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
1,0,0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 2.0]","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[0, 0, 2, 0, 1, 2, 0, 0, 0]",1,0.1,1,0.1,...,0,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
2,0,0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 2.0]","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[1, 2, 1, 0, 2, 2, 0, 2, 2]",0,0.1,1,0.1,...,2,2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
3,0,0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 2.0]","[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 2, 1, 1, 3, 1, 1]",1,0.1,1,0.1,...,1,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
4,0,0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]","[1.0, 2.0]","[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[1, 2, 0, 1, 1, 2, 0, 0, 2]",0,0.1,1,0.1,...,0,2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8639995,29,399,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 3.0]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 0, 2, 2, 2, 0, 4, 0, 1]",0,0.9,5,0.9,...,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
8639996,29,399,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 3.0]","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[0, 2, 1, 3, 1, 0, 2, 0, 0]",1,0.9,5,0.9,...,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
8639997,29,399,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 3.0]","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[1, 2, 1, 3, 3, 1, 2, 2, 0]",0,0.9,5,0.9,...,2,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
8639998,29,399,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 3.0]","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 1, 2, 2, 0, 3, 2, 1, 4]",1,0.9,5,0.9,...,1,4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0


In [18]:
list(df.columns)

['sim_id',
 'respondent_id',
 'context',
 'context_og',
 'profile',
 'profile_og',
 'selected',
 'bjs_spar',
 'bjs_mul',
 'bzs_spar',
 'bzs_mul',
 'sigma_r',
 'bjs',
 'bzs',
 'gender0',
 'gender1',
 'case_note0',
 'case_note1',
 'case_note2',
 'race0',
 'race1',
 'race2',
 'length0',
 'length1',
 'length2',
 'length3',
 'age0',
 'age1',
 'age2',
 'age3',
 'setting0',
 'setting1',
 'setting2',
 'setting3',
 'goal0',
 'goal1',
 'goal2',
 'goal3',
 'goal4',
 'n_removal0',
 'n_removal1',
 'n_removal2',
 'reason0',
 'reason1',
 'reason2',
 'reason3',
 'reason4',
 'gender',
 'case_note',
 'race',
 'length',
 'age',
 'setting',
 'goal',
 'n_removal',
 'reason',
 'has_social0',
 'has_social1',
 'question_bucket0',
 'question_bucket1',
 'question_bucket2',
 'question_bucket3',
 'has_social',
 'question_bucket']

In [71]:
df_to_json=df[['bjs_spar','bjs_mul','bzs_spar','bzs_mul','sigma_r','bjs','bzs']]
df_to_json

,bjs_spar,bjs_mul,bzs_spar,bzs_mul,sigma_r,bjs,bzs
0,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [..."
1,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [..."
2,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [..."
3,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [..."
4,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [..."
...,...,...,...,...,...,...,...
8639995,0.9,5,0.9,5,4.923496,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0..."
8639996,0.9,5,0.9,5,4.923496,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0..."
8639997,0.9,5,0.9,5,4.923496,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0..."
8639998,0.9,5,0.9,5,4.923496,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0..."


In [73]:
def remove_first_level_bjs(base_bj, test_att):
    acc=[]
    count=0
    for i in [x for x in test_att.values()]:
        acc.append(count)
        
        count+=i
    base_bj=np.delete(base_bj,acc)
    return base_bj
def aflb(pu):
    return remove_first_level_bjs(pu['bjs'],test_att)
df_to_json['bjs_24']=df_to_json.apply(aflb,axis=1)

KeyboardInterrupt: 

In [74]:
df_to_json['bjs_24']

0          [-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1...
1          [-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1...
2          [-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1...
3          [-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1...
4          [-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1...
                                 ...                        
8639995    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8639996    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8639997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8639998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8639999    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: bjs_24, Length: 8640000, dtype: object

In [21]:
df_trim=df.drop(['bjs_spar','bjs_mul','bzs_spar','bzs_mul','sigma_r','bjs','bzs'],axis=1) #df w/o repetitive info

In [131]:
one_sim=df_trim[:72*4000]
one_sim.to_csv('one_trim.csv')

In [122]:
df_trim.to_csv('trim.csv')

In [75]:
bjs24=df_to_json[::72*4000]['bjs_24']
df_bjs24= pd.DataFrame(bjs24.to_numpy())
t=[]
for i in df_bjs24[0]: #df to series
    t.append(i)
pd.DataFrame(t)
df_bjss=pd.DataFrame(t,columns=['gender1','case_note1','case_note_2','race1','race2','length1','length2','length3','age1','age2','age3',
                                                 'setting1','setting2','setting3','goal1','goal2','goal3','goal4','n_removal1','n_removal2','reason1','reason2','reason3','reason4'])
df_bjss.to_csv('bjss_new.csv')

In [77]:
#df_bjss

In [41]:
def remove_first_level_context(bz,test_con):
    acc=[]
    count =0
    temp=[]
    for i in [x for x in test_con.values()]:
        acc.append(count)
        count+=i
    #bz=np.delete(bz,acc)
    for i in bz:
        temp.append(np.delete(i,acc))
    return temp

In [104]:
def aflbz(pu):
    return remove_first_level_context(pu['bzs_96_t'],test_con_list)
#df_to_json['bzs_96']=df_to_json.apply(aflbz,axis=1)

In [57]:
def flat_bzs(pu):
    #pu=np.array(pu)
    return np.array(pu['bzs_96']).flatten()

In [111]:
df_to_json_s=df_to_json[::72*4000]
df_to_json_s

,bjs_spar,bjs_mul,bzs_spar,bzs_mul,sigma_r,bjs,bzs,bjs_24
0,0.1,1,0.1,1,0.748342,"[0, -2, 0, 0, -1, 0, 1, -2, 0, 0, 2, 1, 0, 2, ...","[[0, 1, 0, 2, -2, 0], [0, -1, 0, 2, -1, -1], [...","[-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1..."
288000,0.1,1,0.1,1,0.17896,"[0, 1, 0, -1, -2, 0, 2, -1, 0, -1, -1, -1, 0, ...","[[0, 2, 0, -1, 2, 2], [0, -2, 0, 2, 1, 2], [0,...","[1, -1, -2, 2, -1, -1, -1, -1, -2, 1, 0, 0, -1..."
576000,0.1,1,0.1,1,0.146496,"[0, -1, 0, -1, 2, 0, -2, 2, 0, 1, 1, 2, 0, 0, ...","[[0, 2, 0, 2, 1, 0], [0, -2, 0, 1, 1, 2], [0, ...","[-1, -1, 2, -2, 2, 1, 1, 2, 0, -1, -1, -1, 1, ..."
864000,0.1,1,0.1,1,0.507031,"[0, -1, 0, 2, -1, 0, -2, 2, 0, -2, -2, 2, 0, -...","[[0, 2, 0, 0, -2, -1], [0, 2, 0, 2, 2, 0], [0,...","[-1, 2, -1, -2, 2, -2, -2, 2, -2, -1, 2, 2, -1..."
1152000,0.1,1,0.1,1,0.41562,"[0, 0, 0, -2, -2, 0, 1, -1, 0, -2, 2, -1, 0, 1...","[[0, 1, 0, -2, -1, 1], [0, -1, 0, -1, 2, 2], [...","[0, -2, -2, 1, -1, -2, 2, -1, 1, -2, -2, -2, -..."
1440000,0.1,1,0.1,1,0.171856,"[0, -2, 0, -1, -2, 0, 1, 0, 0, -1, 1, -2, 0, -...","[[0, 1, 0, 0, -2, -1], [0, 2, 0, 0, -2, -1], [...","[-2, -1, -2, 1, 0, -1, 1, -2, -2, 2, -1, -2, 2..."
1728000,0.1,1,0.1,1,0.185276,"[0, 1, 0, 1, 2, 0, 1, -1, 0, 1, -1, -2, 0, 1, ...","[[0, 2, 0, -1, 0, -2], [0, 2, 0, 2, 0, 0], [0,...","[1, 1, 2, 1, -1, 1, -1, -2, 1, 1, 2, 2, 2, -2,..."
2016000,0.1,1,0.1,1,0.076368,"[0, 2, 0, 1, -1, 0, -2, 1, 0, 1, 1, 2, 0, 0, 1...","[[0, -2, 0, 2, 0, 2], [0, 2, 0, -1, 2, -1], [0...","[2, 1, -1, -2, 1, 1, 1, 2, 0, 1, 2, 2, -2, -2,..."
2304000,0.1,1,0.1,1,0.635129,"[0, 1, 0, 0, 2, 0, 0, -1, 0, -2, -1, 2, 0, -1,...","[[0, 2, 0, 0, 2, -1], [0, -1, 0, 2, 0, 2], [0,...","[1, 0, 2, 0, -1, -2, -1, 2, -1, -2, -1, -1, -2..."
2592000,0.1,1,0.1,1,0.906625,"[0, 1, 0, 2, -2, 0, -1, -2, 0, -1, 0, -2, 0, 1...","[[0, -1, 0, -1, 1, -1], [0, -1, 0, 0, 2, 1], [...","[1, 2, -2, -1, -2, -1, 0, -2, 1, -2, -2, -2, 1..."


In [113]:
df_to_json_s['bzs_96']=df_to_json_s.apply(aflbz,axis=1)


<ipython-input-113-a0883f6b1d7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_json_s['bzs_96']=df_to_json_s.apply(aflbz,axis=1)


In [99]:
def aflbt(pu):
    return remove_first_level_bzs(pu['bzs'],test_att)

In [97]:
#not neccesary since the first levels of bjs are zeros already, but it helps with matching coefficients in R
def remove_first_level_bzs(base_bz, test_att): 
    acc=[]
    count=0
    for i in [x for x in test_att.values()]:
        acc.append(count)
        
        count+=i
    base_bz=np.delete(base_bz,acc,axis=0)
    return base_bz

In [98]:
#np.delete(df_to_json_s['bzs'][0],0,axis=0)

In [112]:
df_to_json_s['bzs_96_t']=df_to_json_s.apply(aflbt,axis=1)

<ipython-input-112-93627c8eb95b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_json_s['bzs_96_t']=df_to_json_s.apply(aflbt,axis=1)


In [118]:
#df_to_json_s['bzs'][0]

In [116]:
df_to_json_s['bzs_96_t'][0]

array([[ 0, -1,  0,  2, -1, -1],
       [ 0,  2,  0, -2,  0,  1],
       [ 0, -2,  0, -1, -1,  1],
       [ 0,  0,  0, -1,  0,  1],
       [ 0,  2,  0,  2, -2,  1],
       [ 0, -2,  0,  1, -1, -1],
       [ 0,  2,  0, -2, -2,  1],
       [ 0,  2,  0, -1, -2,  1],
       [ 0, -1,  0,  1,  2, -1],
       [ 0,  2,  0,  2, -2, -2],
       [ 0,  2,  0, -1,  2,  2],
       [ 0, -2,  0,  0,  2, -1],
       [ 0, -2,  0,  2,  0,  2],
       [ 0,  1,  0,  1,  1, -2],
       [ 0,  2,  0, -1, -2,  1],
       [ 0,  1,  0,  2,  2, -1],
       [ 0,  1,  0,  1, -2, -1],
       [ 0, -1,  0,  1,  0,  2],
       [ 0, -2,  0,  2,  2, -2],
       [ 0,  2,  0, -2,  1,  1],
       [ 0,  2,  0,  1, -2,  1],
       [ 0, -1,  0, -2,  2, -2],
       [ 0, -2,  0,  2, -2,  2],
       [ 0, -2,  0, -2,  0,  2]])

In [114]:
df_to_json_s['bzs_96_f']=df_to_json_s.apply(flat_bzs,axis=1)

<ipython-input-114-01e15528de63>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_json_s['bzs_96_f']=df_to_json_s.apply(flat_bzs,axis=1)


In [119]:
df_to_json_s['bzs_96_f'][0]

array([-1,  2, -1, -1,  2, -2,  0,  1, -2, -1, -1,  1,  0, -1,  0,  1,  2,
        2, -2,  1, -2,  1, -1, -1,  2, -2, -2,  1,  2, -1, -2,  1, -1,  1,
        2, -1,  2,  2, -2, -2,  2, -1,  2,  2, -2,  0,  2, -1, -2,  2,  0,
        2,  1,  1,  1, -2,  2, -1, -2,  1,  1,  2,  2, -1,  1,  1, -2, -1,
       -1,  1,  0,  2, -2,  2,  2, -2,  2, -2,  1,  1,  2,  1, -2,  1, -1,
       -2,  2, -2, -2,  2, -2,  2, -2, -2,  0,  2])

In [44]:
df_to_json_s['bzs'][0]

array([[ 0,  1,  0,  2, -2,  0],
       [ 0, -1,  0,  2, -1, -1],
       [ 0,  0,  0, -2,  0,  1],
       [ 0,  2,  0, -2,  0,  1],
       [ 0, -2,  0, -1, -1,  1],
       [ 0, -1,  0,  1, -1,  2],
       [ 0,  0,  0, -1,  0,  1],
       [ 0,  2,  0,  2, -2,  1],
       [ 0, -2,  0,  1,  2,  1],
       [ 0, -2,  0,  1, -1, -1],
       [ 0,  2,  0, -2, -2,  1],
       [ 0,  2,  0, -1, -2,  1],
       [ 0,  0,  0,  2,  1, -2],
       [ 0, -1,  0,  1,  2, -1],
       [ 0,  2,  0,  2, -2, -2],
       [ 0,  2,  0, -1,  2,  2],
       [ 0, -2,  0,  1,  0, -1],
       [ 0, -2,  0,  0,  2, -1],
       [ 0, -2,  0,  2,  0,  2],
       [ 0,  1,  0,  1,  1, -2],
       [ 0, -2,  0,  2, -2,  1],
       [ 0,  2,  0, -1, -2,  1],
       [ 0,  1,  0,  2,  2, -1],
       [ 0,  1,  0,  1, -2, -1],
       [ 0, -1,  0,  1,  0,  2],
       [ 0,  0,  0,  1, -1, -2],
       [ 0, -2,  0,  2,  2, -2],
       [ 0,  2,  0, -2,  1,  1],
       [ 0, -1,  0, -2,  0,  1],
       [ 0,  2,  0,  1, -2,  1],
       [ 0

In [45]:
df_to_json_s['bzs_96'][0]

[array([ 1,  2, -2,  0]),
 array([-1,  2, -1, -1]),
 array([ 0, -2,  0,  1]),
 array([ 2, -2,  0,  1]),
 array([-2, -1, -1,  1]),
 array([-1,  1, -1,  2]),
 array([ 0, -1,  0,  1]),
 array([ 2,  2, -2,  1]),
 array([-2,  1,  2,  1]),
 array([-2,  1, -1, -1]),
 array([ 2, -2, -2,  1]),
 array([ 2, -1, -2,  1]),
 array([ 0,  2,  1, -2]),
 array([-1,  1,  2, -1]),
 array([ 2,  2, -2, -2]),
 array([ 2, -1,  2,  2]),
 array([-2,  1,  0, -1]),
 array([-2,  0,  2, -1]),
 array([-2,  2,  0,  2]),
 array([ 1,  1,  1, -2]),
 array([-2,  2, -2,  1]),
 array([ 2, -1, -2,  1]),
 array([ 1,  2,  2, -1]),
 array([ 1,  1, -2, -1]),
 array([-1,  1,  0,  2]),
 array([ 0,  1, -1, -2]),
 array([-2,  2,  2, -2]),
 array([ 2, -2,  1,  1]),
 array([-1, -2,  0,  1]),
 array([ 2,  1, -2,  1]),
 array([-1, -2,  2, -2]),
 array([-2,  2, -2,  2]),
 array([-2, -2,  0,  2])]

In [47]:
#df_bjss

In [66]:
len(df_to_json_s['bzs_96_f'][0])

132

In [62]:
df_bjs24

,0
0,"[-2, 0, -1, 1, -2, 0, 2, 1, 2, -2, 1, 2, -2, 1..."
1,"[1, -1, -2, 2, -1, -1, -1, -1, -2, 1, 0, 0, -1..."
2,"[-1, -1, 2, -2, 2, 1, 1, 2, 0, -1, -1, -1, 1, ..."
3,"[-1, 2, -1, -2, 2, -2, -2, 2, -2, -1, 2, 2, -1..."
4,"[0, -2, -2, 1, -1, -2, 2, -1, 1, -2, -2, -2, -..."
5,"[-2, -1, -2, 1, 0, -1, 1, -2, -2, 2, -1, -2, 2..."
6,"[1, 1, 2, 1, -1, 1, -1, -2, 1, 1, 2, 2, 2, -2,..."
7,"[2, 1, -1, -2, 1, 1, 1, 2, 0, 1, 2, 2, -2, -2,..."
8,"[1, 0, 2, 0, -1, -2, -1, 2, -1, -2, -1, -1, -2..."
9,"[1, 2, -2, -1, -2, -1, 0, -2, 1, -2, -2, -2, 1..."


In [120]:
bzs96_f=df_to_json_s['bzs_96_f']
df_bzs96_f= pd.DataFrame(bzs96_f.to_numpy())
t=[]
for i in df_bzs96_f[0]: #df to series
    t.append(i)
pd.DataFrame(t)
df_bzss=pd.DataFrame(t)
#df_bjss.to_csv('bjss_new.csv')

In [123]:
df_bzss.to_csv('bzss.csv')

In [121]:
df_bzss

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-1,2,-1,-1,2,-2,0,1,-2,-1,...,2,-2,-2,2,-2,2,-2,-2,0,2
1,-2,2,1,2,-2,-2,0,2,-1,-1,...,-2,0,-2,0,-2,2,2,-2,2,1
2,-2,1,1,2,0,2,1,-1,2,0,...,0,2,0,-1,1,-1,2,-2,1,1
3,2,2,2,0,1,1,-2,-2,-2,-2,...,1,1,1,1,1,-1,-2,1,-2,1
4,-1,-1,2,2,-1,2,2,1,-2,-2,...,2,-1,-1,0,-2,-1,0,1,1,2
5,2,0,-2,-1,2,-1,-2,-1,1,0,...,2,-1,2,-2,2,-1,-1,-2,0,0
6,2,2,0,0,0,1,-1,2,1,-2,...,0,1,0,2,1,1,1,1,-1,-2
7,2,-1,2,-1,-1,2,-1,1,-1,-2,...,-1,1,1,-1,-2,-1,1,-2,-1,-1
8,-1,2,0,2,-2,2,-2,-1,-1,1,...,2,-2,0,2,-2,1,-2,1,-1,-2
9,-1,0,2,1,-2,-2,2,2,-1,1,...,-1,-1,-2,-1,1,2,2,2,2,2
